In [ ]:
# carregar dataset rudementar
# ==============================================================
# CARREGAR DATASET RUDIMENTAR — Coleta de POIs do OSM
# Angola (Pontos Turísticos com coordenadas conhecidas)
# ==============================================================

import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pathlib import Path
import time

# === Diretório de saída ===
DATA_DIR = Path("../data/geoespacial")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# === Locais turísticos (com coordenadas conhecidas) ===
locais_turisticos = [
    {"nome": "Quedas de Calandula", "lat": -9.075025, "lon": 16.001131, "provincia": "Malanje"},
    {"nome": "Miradouro da Lua", "lat": -9.221147, "lon": 13.090001, "provincia": "Luanda"},
    {"nome": "Museu Kulumbimbi", "lat": -6.264389, "lon": 14.245581, "provincia": "Zaire"},
    {"nome": "Reserva Parcial do Namibe", "lat": -15.766760, "lon": 12.399914, "provincia": "Namibe"},
    {"nome": "Fortaleza de São Miguel", "lat": -8.808343416048587, "lon": 13.223444621289627, "provincia": "Luanda"},
    {"nome": "Ilha do Mussulo", "lat": -8.964635, "lon":13.050462, "provincia": "Luanda"},
    {"nome": "Cânion do Rio Kalandula", "lat": -9.074959, "lon": 16.001378, "provincia": "Malanje"},
    {"nome": "Parque Nacional da Quiçama", "lat": -9.750007, "lon": 13.583333, "provincia": "Luanda"},
    {"nome": "Cristo Rei", "lat": -14.940039, "lon": 13.511860, "provincia": "Huambo"},
    {"nome": "Serra da Leba", "lat": -15.0788, "lon": 13.2397, "provincia": "Huíla"},
    {"nome": "Fenda da Tundavala", "lat": -14.817465, "lon": 13.381539, "provincia": "Huíla"},
    {"nome": "Baía Azul", "lat": -12.627978, "lon": 13.234185, "provincia": "Benguela"},
    {"nome": "Quedas do Rio Chiumbe", "lat": -11.021380, "lon": 20.203180, "provincia": "Moxico"},
    {"nome": "Parque Nacional de Cameia", "lat": -11.883338, "lon": 21.666681, "provincia": "Moxico"},
    {"nome": "Grutas do Nzenzo", "lat": -7.520192, "lon":14.565103, "provincia": "Uíge"}
]

BUFFER_RADIUS = 5000  # 5 km
dfs = []

# === Tags OSM — Turismo + Infraestrutura ===
TAGS = {
    "tourism": True,
    "amenity": True,
    "leisure": True,
    "natural": True,
    "landuse": True,
    "highway": True,
    "waterway": True,
    "power": True,
    "man_made": True,
    "railway": True,
    "building": True
}


def coletar_pois(lat, lon, tags, buffer_m):
    """Retorna os POIs dentro do raio (buffer_m) ao redor das coordenadas."""
    try:
        gdf_ponto = gpd.GeoDataFrame([["ponto", Point(lon, lat)]],
                                     columns=["nome", "geometry"],
                                     crs="EPSG:4326")
        area_busca = gdf_ponto.to_crs(3857).buffer(buffer_m).to_crs(4326).iloc[0]

        pois = ox.features_from_polygon(area_busca, tags).reset_index()
        if pois.empty:
            return None

        # Garantir colunas
        colunas_necessarias = [
            "osm_id", "name", "tourism", "amenity", "leisure",
            "natural", "landuse", "highway", "waterway",
            "power", "man_made", "railway", "building", "geometry"
        ]
        for c in colunas_necessarias:
            if c not in pois.columns:
                pois[c] = None
        return pois[colunas_necessarias]
    except Exception as e:
        print(f" Erro na coleta: {e}")
        return None


# === Loop pelos locais ===
for item in locais_turisticos:
    nome = item["nome"]
    lat = item["lat"]
    lon = item["lon"]
    prov = item["provincia"]

    print(f"\n Coletando POIs para {nome} ({prov})...")

    pois = coletar_pois(lat, lon, TAGS, BUFFER_RADIUS)

    if pois is not None:
        # Adicionar metadados
        pois["ponto_turistico"] = nome
        pois["provincia"] = prov
        pois["fonte_dados"] = "OpenStreetMap"
        pois["latitude"] = pois.to_crs(3857).geometry.centroid.to_crs(4326).y
        pois["longitude"] = pois.to_crs(3857).geometry.centroid.to_crs(4326).x

        # Salvar individualmente
        output_path = DATA_DIR / f"pois_{nome.replace(' ', '_').lower()}.geojson"
        pois.to_file(output_path, driver="GeoJSON")
        print(f"    Salvo: {output_path}")

        dfs.append(pois)
    else:
        print(f" Nenhum POI encontrado em {nome}.")

    time.sleep(1.5)  # respeitar limite de requisições

# === Combinar tudo ===
if dfs:
    pois_final = gpd.GeoDataFrame(pd.concat(dfs, ignore_index=True), crs="EPSG:4326")
    output_final = DATA_DIR / "pontos_turisticos_angola_raw.geojson"
    pois_final.to_file(output_final, driver="GeoJSON")
    print(f"\n Arquivo combinado salvo em: {output_final}")
else:
    print("\n Nenhum POI foi coletado.")



📍 Coletando POIs para Quedas de Calandula (Malanje)...
   💾 Salvo: ..\data\geoespacial\pois_quedas_de_calandula.geojson

📍 Coletando POIs para Miradouro da Lua (Luanda)...
   💾 Salvo: ..\data\geoespacial\pois_miradouro_da_lua.geojson

📍 Coletando POIs para Museu Kulumbimbi (Zaire)...
   💾 Salvo: ..\data\geoespacial\pois_museu_kulumbimbi.geojson

📍 Coletando POIs para Reserva Parcial do Namibe (Namibe)...
   💾 Salvo: ..\data\geoespacial\pois_reserva_parcial_do_namibe.geojson

📍 Coletando POIs para Fortaleza de São Miguel (Luanda)...
   💾 Salvo: ..\data\geoespacial\pois_fortaleza_de_são_miguel.geojson

📍 Coletando POIs para Ilha do Mussulo (Luanda)...
   💾 Salvo: ..\data\geoespacial\pois_ilha_do_mussulo.geojson

📍 Coletando POIs para Cânion do Rio Kalandula (Malanje)...
   💾 Salvo: ..\data\geoespacial\pois_cânion_do_rio_kalandula.geojson

📍 Coletando POIs para Parque Nacional da Quiçama (Luanda)...
 Erro na coleta: No matching features. Check query location, tags, and log.
 Nenhum POI e

In [9]:
# Normalizar 

# ==============================================================
# NORMALIZAÇÃO DOS DADOS GEOFÍSICOS E DE INFRAESTRUTURA
# ==============================================================

import geopandas as gpd
from pathlib import Path

DATA_DIR = Path("../data/geoespacial")
input_path = DATA_DIR / "pontos_turisticos_angola_raw.geojson"
output_path = DATA_DIR / "pontos_turisticos_angola_normalizado.geojson"

# ===  Carregar dados brutos ===
pois = gpd.read_file(input_path)

# ===  Limpeza básica ===
if "osm_id" in pois.columns:
    pois = pois.drop_duplicates(subset=["osm_id"], keep="first")
else:
    pois = pois.drop_duplicates(subset=["name", "geometry"], keep="first")

pois = pois[pois.is_valid]
pois = pois.dropna(subset=["geometry"])

# ===  Garantir colunas críticas ===
for col in ["name", "tourism", "amenity", "leisure", "natural", "landuse", 
            "highway", "power", "latitude", "longitude"]:
    if col not in pois.columns:
        pois[col] = None

# ===  Correção e normalização ===
pois["name"] = pois["name"].fillna("Desconhecido").astype(str)

map_tourism = {
    "guest_house": "accommodation",
    "hotel": "accommodation",
    "motel": "accommodation",
    "museum": "culture",
    "attraction": "attraction"
}
map_amenity = {
    "restaurant": "food_service",
    "fast_food": "food_service",
    "hospital": "health",
    "school": "education"
}

pois["tourism"] = pois["tourism"].replace(map_tourism)
pois["amenity"] = pois["amenity"].replace(map_amenity)

# ===  Variáveis derivadas ===
pois["is_touristic"] = pois["tourism"].notna().astype(int)
pois["is_infrastructure"] = pois[["amenity", "highway", "power"]].notna().any(axis=1).astype(int)

# ===  Salvar resultado ===
pois.to_file(output_path, driver="GeoJSON")
print(f" Dados normalizados salvos em: {output_path}")




 Dados normalizados salvos em: ..\data\geoespacial\pontos_turisticos_angola_normalizado.geojson


In [ ]:
# ==============================================================
# NORMALIZAÇÃO DOS DADOS CLIMATICOS-AMBIENTAIS
# ==============================================================

In [ ]:
# ==============================================================
# NORMALIZAÇÃO DOS DADOS ECONÔMICOS-SOCIAIS
# ==============================================================

In [11]:
# Gerar variáveis espaciais 
# ============================================================== 
# GERAR VARIÁVEIS ESPACIAIS
# ==============================================================

import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from pathlib import Path
import numpy as np

DATA_DIR = Path("../data/geoespacial")
input_path = DATA_DIR / "pontos_turisticos_angola_normalizado.geojson"
output_path = DATA_DIR / "pontos_turisticos_angola_features.geojson"

# === Carregar dados normalizados ===
pois = gpd.read_file(input_path)

# === Converter para projeção métrica (metros) para cálculos espaciais ===
pois = pois.to_crs(epsg=3857)

# === Exemplo 1: Calcular centroid de cada ponto turístico (se houver polígonos) ===
# (No caso de pontos simples, é só o próprio ponto)
pois['centroid_x'] = pois.geometry.centroid.x
pois['centroid_y'] = pois.geometry.centroid.y

# === Exemplo 2: Distância até o ponto turístico mais próximo ===
# Criar uma função para calcular a menor distância entre cada ponto
def nearest_distance(point, others):
    # Retorna a menor distância (em metros) até outro ponto
    distances = others.geometry.distance(point)
    distances = distances[distances > 0]  # ignora distância zero (o próprio ponto)
    if len(distances) > 0:
        return distances.min()
    else:
        return np.nan

pois['dist_to_nearest_poi'] = pois.geometry.apply(lambda p: nearest_distance(p, pois))

# === Exemplo 3: Buffer / contagem de POIs próximos (densidade) ===
BUFFER_RADIUS = 1000  # metros
pois['pois_within_1km'] = pois.geometry.apply(lambda g: pois[pois.geometry.within(g.buffer(BUFFER_RADIUS))].shape[0] - 1) 
# -1 para excluir o próprio ponto

# === Exemplo 4: Flag de proximidade a categorias específicas ===
# Por exemplo, se existe hospital ou escola em 1km
def has_category_nearby(point, category_col, category_value, radius=1000):
    buffer = point.buffer(radius)
    subset = pois[(pois[category_col] == category_value) & (pois.geometry.within(buffer))]
    return int(len(subset) > 0)

pois['hospital_nearby'] = pois.geometry.apply(lambda g: has_category_nearby(g, 'amenity', 'hospital'))
pois['school_nearby'] = pois.geometry.apply(lambda g: has_category_nearby(g, 'amenity', 'school'))

# === Salvar dataset com variáveis espaciais ===
pois = pois.to_crs(epsg=4326)  # voltar para lat/lon
pois.to_file(output_path, driver="GeoJSON")
print(f" Variáveis espaciais geradas e salvas em: {output_path}")


 Variáveis espaciais geradas e salvas em: ..\data\geoespacial\pontos_turisticos_angola_features.geojson


In [ ]:
# Cruzar camadas (join espacial) 


In [ ]:
# Salvar dataset final 

In [ ]:
# Entrega esperada: model_inputs.csv